In [ ]:
import vcdvcd
import matplotlib.pyplot as plt
import numpy as np
from dataclasses import dataclass

In [ ]:
def condition_signal_tv(tv):
    x, y = list(zip(*tv))
    if not isinstance(y[0], int):
        make_bin_to_int = lambda x : int(x, base=2)
        #x = map(make_bin_to_int, x)
        y = list(map(make_bin_to_int, y))
    tv = list(zip(x, y))
    return tv

def expand_signal(xin, yin, length=None):
    import numpy as np
    import pandas as pd
    xin = list(xin)
    if not length:
        length = max(xin)+1

    # Select all points less than length, then add last point, extrapolting that the last point has no transitions
    # This will always result in an accurate frame of the desired length
    df = pd.DataFrame({"x": xin, "y": yin})
    df = df[df["x"] < length]
    df.loc[len(df)] = {'x': length-1, "y": list(df["y"])[-1]}

    # At the transition add a double x point at the initial value before the change index
    dout = {"x": [], "y": []}
    for thisr, nextr in zip(df.iterrows(), list(df.iterrows())[1:]):
        _, next = nextr
        _, this = thisr
        dout["x"].append(this["x"])
        dout["y"].append(this["y"])

        dout["x"].append(next["x"])
        dout["y"].append(this["y"])
    return dout['x'], dout['y']

def condition_expand(signal, endtime: float):
    """Condition and expand VCD signal to endtime."""
    signal.tv = condition_signal_tv(signal.tv)
    x, y = list(zip(*signal.tv))
    x, y = expand_signal(x, y, endtime)
    return np.array(x, dtype=float), np.array(y, dtype=float)
    
@dataclass
class Signal:
    title: str
    x: np.array
    y: np.array
    xscale: float
    endtime: float

    @classmethod
    def load(cls, vcd: vcdvcd.VCDVCD, signal_id: str, title: str, endtime: float):
        xscale = float(vcd.timescale["timescale"])
        x, y = condition_expand(vcd[signal_id], endtime)
        return cls(title=title, x=np.asarray(x), y=np.asarray(y), xscale=xscale, endtime=endtime)

In [ ]:
def get_signals(fname="../hdl/q2/logs/vlt_dump.vcd"):
    vcd = vcdvcd.VCDVCD(fname)
    signal_tags = (
        ("TOP.clk", "clk"),
        ("TOP.rstn", "rstn"),
        ("TOP.x", "x"),
        ("TOP.z_moore", "Moore"),
        ("TOP.z_mealy", "Mealy")
    )
    
    endtime = max(vcd[name].endtime for name, _ in signal_tags)

    signals: List[Signal] = [
        Signal.load(vcd=vcd, signal_id=signal_id, title=title, endtime=endtime) 
        for signal_id, title in signal_tags]
    return signals

def plot_timing_diagram(signals):
    '''
    Normalize signals and space them out on one axis.
    '''
    fig, ax = plt.subplots(figsize=(12, 6))
    spacing = 2.2

    clk_signal = next(sig for sig in signals if sig.title == "clk")
    mealy_signal = next(sig for sig in signals if sig.title == "Mealy")
    moore_signal = next(sig for sig in signals if sig.title == "Moore")

    offsets = {}
    # Digital signals
    for i, signal in enumerate(signals):
        y_offset = -i * spacing
        offsets[signal.title] = y_offset  # store offset for later reference

        endtime = signal.endtime
        xscale = signal.xscale
        x = signal.x*xscale*1e9
        
        y_range = (max(signal.y)-min(signal.y))
        y = np.zeros_like(signal.y) if y_range == 0 else signal.y/y_range
    
        label = signal.title

        # Normalize: each digital waveform ranges from baseline (y_offset) to y_offset+1
        ax.step(x, y_offset + y, where="post", color="black", lw=1.5)
        ax.hlines(y_offset, 0, endtime * xscale, color="0.8",
                  lw=0.5, linestyle="--")  # baseline
        ax.text(-1e-3, y_offset + 0.5, label, ha="right", va="center",
                fontsize=12, weight="bold")

    clk_x = clk_signal.x * clk_signal.xscale * 1e9
    clk_y = clk_signal.y

    ## color Mealy
    for i in range(len(clk_x) - 1):
        if clk_y[i] == 0 and ((clk_x[i] > 0.01 and clk_x[i] < 0.15) or (clk_x[i] > 0.17 and clk_x[i] < 0.3)):
            x0, x1 = clk_x[i], clk_x[i+1]
            ax.axvspan(x0, x1, ymin=0.090, ymax=0.175,
                       facecolor="lightblue", alpha=0.8)

    # color moore
    prev_clk = 1
    count = 0
    color = "lightblue"
    for i in range(len(clk_x) - 1):
        if (clk_x[i] > 0.015 and clk_x[i] < 0.16) or (clk_x[i] > 0.175 and clk_x[i] < 0.31):
            x0, x1 = clk_x[i], clk_x[i+1]
            if (clk_y[i] != prev_clk):
                prev_clk = clk_y[i]
                count += 1
                if (count == 2):
                    count = 0
                    if (color == "lightblue"):
                        color = "lightcoral"
                    else:
                        color = "lightblue"
            ax.axvspan(x0, x1, ymin=0.275, ymax=0.355,
                       facecolor=color, alpha=0.8)

    # Formatting
    ax.set_ylim(-spacing*(len(signals)-1) - spacing/2, spacing)
    ax.set_xlabel("Time (ns)", fontsize=12)
    ax.set_yticks([])
    ax.set_title("Moore and Mealy Message Converter", fontsize=14, weight="bold")

    # Gridlines
    ax.xaxis.set_major_locator(plt.MultipleLocator(0.10))
    ax.xaxis.set_minor_locator(plt.MultipleLocator(0.01))
    ax.grid(axis="x", which="minor", linestyle=":", color="0.7")
    ax.grid(axis="x", which="major", linestyle=":", color="0.7")

    return fig, [ax]
    
signals = get_signals()
fig, axes = plot_timing_diagram(list(signals))
axes[0].set_xlim(0, 0.315)
fig.savefig("../report/assets/q2_wave.pdf", bbox_inches="tight")
plt.show()


In [ ]:
def get_signals(fname="../hdl/q3/logs/vlt_dump.vcd"):
    vcd = vcdvcd.VCDVCD(fname)
    signal_tags = (
        ("TOP.clk", "clk"),
        ("TOP.rstn", "rstn"),
        ("TOP.serdata", "serdata"),
        ("TOP.valid", "valid")
    )
    
    endtime = max(vcd[name].endtime for name, _ in signal_tags)

    signals: List[Signal] = [
        Signal.load(vcd=vcd, signal_id=signal_id, title=title, endtime=endtime) 
        for signal_id, title in signal_tags]
    return signals

def plot_timing_diagram(signals):
    '''
    Normalize signals and space them out on one axis.
    '''
    fig, ax = plt.subplots(figsize=(12, 6))
    spacing = 2.2

    clk_signal = next(sig for sig in signals if sig.title == "clk")
    valid_signal = next(sig for sig in signals if sig.title == "valid")

    offsets = {}
    # Digital signals
    for i, signal in enumerate(signals):
        y_offset = -i * spacing
        offsets[signal.title] = y_offset  # store offset for later reference

        endtime = signal.endtime
        xscale = signal.xscale
        x = signal.x*xscale*1e9
        
        y_range = (max(signal.y)-min(signal.y))
        y = np.zeros_like(signal.y) if y_range == 0 else signal.y/y_range
    
        label = signal.title

        # Normalize: each digital waveform ranges from baseline (y_offset) to y_offset+1
        ax.step(x, y_offset + y, where="post", color="black", lw=1.5)
        ax.hlines(y_offset, 0, endtime * xscale, color="0.8",
                  lw=0.5, linestyle="--")  # baseline
        ax.text(-1e-3, y_offset + 0.5, label, ha="right", va="center",
                fontsize=12, weight="bold")

    clk_x = clk_signal.x * clk_signal.xscale * 1e9
    clk_y = clk_signal.y
    valid_x = valid_signal.x * valid_signal.xscale * 1e9
    valid_y = valid_signal.y

    for i in range(len(valid_x) - 1):
        if valid_y[i] == 1:
            x0, x1 = valid_x[i], valid_x[i+1]
            ax.axvspan(x0, x1, ymin=0.110, ymax=0.210,
                       facecolor="lightblue", alpha=0.8)

    # Formatting
    ax.set_ylim(-spacing*(len(signals)-1) - spacing/2, spacing)
    ax.set_xlabel("Time (ns)", fontsize=12)
    ax.set_yticks([])
    ax.set_title("Serial Communication Sequence Detector", fontsize=14, weight="bold")

    # Gridlines
    ax.xaxis.set_major_locator(plt.MultipleLocator(0.16, offset=-0.005))
    ax.xaxis.set_minor_locator(plt.MultipleLocator(0.08, offset=-0.005))
    ax.grid(axis="x", which="minor", linestyle=":", color="0.7")
    ax.grid(axis="x", which="major", linestyle=":", color="0.7")

    return fig, [ax]
    
signals = get_signals()
fig, axes = plot_timing_diagram(list(signals))
axes[0].set_xlim(0, 0.81)
fig.savefig("../report/assets/q3_wave.pdf", bbox_inches="tight")
plt.show()
